<a href="https://colab.research.google.com/github/yusuke-satani/swing_classification/blob/main/LSTM_models.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import cv2
import numpy as np
from ultralytics import YOLO
import os

In [ ]:
model = YOLO('yolov8n-pose.pt')

In [ ]:
import torch

def extract_keypoints(frames):
    all_keypoints = []
    device = torch.device('cuda')
    for frame in frames:
        frame = torch.tensor(frame, dtype=torch.float32).to(device)
        results = model(frame)

        if len(results) > 0 and len(results[0].keypoints) > 0:
            keypoints_list = results[0].keypoints
            bboxes = results[0].boxes  # list of bounding boes

            if bboxes is not None and len(bboxes) > 0:
                # calculate area of bounding boxes
                try:
                    xyxy = bboxes.xyxy.cpu().numpy()  # get cordinates of bounding boxes
                    areas = [(box[2] - box[0]) * (box[3] - box[1]) for box in xyxy]
                except IndexError as e:
                    print(f"Error calculating areas: {e}")
                    print(f"Bounding boxes: {bboxes}")
                    continue

                # get the index of the biggest bounding box
                max_area_index = np.argmax(areas)

                # get keypoints of the biggest bounding box
                keypoints = keypoints_list[max_area_index].xy[0].cpu().numpy()

                all_keypoints.append(keypoints)
            else:
                print(f"Warning: No bounding boxes found in frame.")
        else:
            print(f"Warning: No keypoints found in frame.")

    return all_keypoints  # np.arrayの変換を削除


def normalize_keypoints(keypoints):
    hip_index = 11  #  index of left hip
    shoulder_index = 5  # index of shoulder

    normalized_keypoints = []
    for frame_keypoints in keypoints:
        if len(frame_keypoints) > max(hip_index, shoulder_index):
            hip_point = frame_keypoints[hip_index]
            shoulder_point = frame_keypoints[shoulder_index]

            # Calculate the point at the waist as the origin (0,0) and the other points as relative positions to make the model robust
            relative_points = frame_keypoints - hip_point

            scale_factor = np.linalg.norm(shoulder_point - hip_point)
            if scale_factor != 0:
                relative_points /= scale_factor

            normalized_keypoints.append(relative_points)
        else:
            print(f"Warning: Frame with insufficient keypoints detected. Skipping this frame.")

    return np.array(normalized_keypoints)

def process_folder(folder_path, label):
    print(f"Processing folder: {folder_path}")
    video_data = []
    for video_file in os.listdir(folder_path):
        if video_file.endswith(('.mp4', '.avi', '.mov')):
            video_path = os.path.join(folder_path, video_file)
            frames = process_video(video_path)
            keypoints = extract_keypoints(frames)
            if len(keypoints) > 0:
                normalized_keypoints = normalize_keypoints(keypoints)
                if len(normalized_keypoints) > 0:
                    video_data.append((normalized_keypoints, label))
                else:
                    print(f"Warning: No valid keypoints found in video {video_file}")
            else:
                print(f"Warning: No keypoints detected in video {video_file}")
    return video_data

    print(f"Number of videos processed: {len(video_data)}")

shot_types = ['forehand_stroke','forehand_slice','forehand_volley', 'backhand_stroke', 'backhand_volley', 'backhand_slice']
all_data = []
for label, shot_type in enumerate(shot_types):
    folder_path = f'/Users/yusuke.s/Documents/pickleball_videos/{shot_type}'
    all_data.extend(process_folder(folder_path, label))

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import numpy as np
all_data = np.load('/content/drive/MyDrive/all_data.npy', allow_pickle=True)

In [ ]:
def pad_sequences(sequences, max_length):
    # Align sequence lengths
    return [seq[:max_length] if len(seq) > max_length else np.pad(seq, ((0, max_length - len(seq)), (0, 0), (0, 0)), 'constant') for seq in sequences]

X = [data[0] for data in all_data]
y = [data[1] for data in all_data]

# Align sequence lengths
X_padded = pad_sequences(X, 100)

X_array = np.array(X_padded)
y_array = np.array(y)
shot_types = ['forehand_stroke','forehand_volley','forehand_slice','backhand_stroke','backhand_volley','backhand_slice']
y_onehot = np.eye(len(shot_types))[y_array]

In [ ]:
import numpy as np
from tensorflow.keras.utils import Sequence
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, AdditiveAttention, Concatenate
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import Adam
from sklearn.model_selection import train_test_split
from sklearn.utils.class_weight import compute_class_weight

class DataGenerator(Sequence):
    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return int(np.ceil(len(self.x) / float(self.batch_size)))

    def __getitem__(self, idx):
        batch_x = self.x[idx * self.batch_size:(idx + 1) * self.batch_size]
        batch_y = self.y[idx * self.batch_size:(idx + 1) * self.batch_size]
        return batch_x, batch_y

def create_lstm_attention_model(sequence_length, num_keypoints, num_coords, num_classes):
    inputs = Input(shape=(sequence_length, num_keypoints * num_coords))
    x = LSTM(128, return_sequences=True)(inputs)
    attention = AdditiveAttention()([x, x])
    x = Concatenate()([x, attention])
    x = LSTM(64, return_sequences=True)(x)
    attention = AdditiveAttention()([x, x])
    x = Concatenate()([x, attention])
    x = LSTM(32)(x)
    x = Dense(128, activation='relu')(x)
    x = Dropout(0.5)(x)
    x = Dense(64, activation='relu')(x)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)

    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(learning_rate=0.0001), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

X_train, X_test, y_train, y_test = train_test_split(X_array, y_onehot, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], -1)
X_val = X_val.reshape(X_val.shape[0], X_val.shape[1], -1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1)

train_generator = DataGenerator(X_train, y_train, batch_size=16)
val_generator = DataGenerator(X_val, y_val, batch_size=16)
test_generator = DataGenerator(X_test, y_test, batch_size=16)

sequence_length = X_train.shape[1]
num_keypoints = 17
num_coords = 2
num_classes = 6
model = create_lstm_attention_model(sequence_length, num_keypoints, num_coords, num_classes)

class_weights = compute_class_weight('balanced', classes=np.unique(np.argmax(y_train, axis=1)), y=np.argmax(y_train, axis=1))
class_weight_dict = dict(enumerate(class_weights))

early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)


history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=100,
    callbacks=[early_stopping],
    class_weight=class_weight_dict
)

test_loss, test_accuracy = model.evaluate(test_generator)
print(f"Test accuracy: {test_accuracy:.4f}")


In [ ]:
model.save('swing_class.h5')

In [ ]:
from tensorflow.keras.models import load_model

loaded_model = load_model('/content/drive/MyDrive/swing_class.h5')

predictions = loaded_model.predict(test_generator)


In [ ]:
test_generator
np.argmax(predictions, axis=1)